# MoCap Rasp Arena

This notebook is an implementation of the MoCap Rasp Optical Tracking Arena.

---

In [1]:
# Importing modules...
import numpy as np
import socket
import time

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.camera import Camera
from modules.vision.synchronizer import Synchronizer

from modules.integration.client import Client
from modules.integration.mocaprasp.server import MoCapRasp_Server

from intrinsic import * # Intrinsic Calibration data

# Setting Up

All `Camera` and `Client` objects will be instanciated right along with the `Server` object.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Create clients
for K, D in zip(intrinsic_matrices, distortion_coefficients):
    # Generate associated camera model
    camera = (Camera(# Intrinsic Parameters
                     resolution=(960, 720),
                     intrinsic_matrix=K.copy(),
 
                     # Fisheye Lens Distortion Model
                     distortion_model='fisheye',
                     distortion_coefficients=D.copy()
                     ))
    
    clients.append(Client(camera=camera))

# Create server
server = MoCapRasp_Server(clients=clients,
                          server_address=('0.0.0.0', 8888))

# Register clients
server.register_clients()

[SERVER] Wrapping up CoppeliaSim scene info
[SERVER] Scene info sent
[SERVER] Scene set!


# Extrinsic Calibration

The Cameras' Extrinsic Parameters will be estimated. For that, a calibration routine will be requested and the data will be post-processed for the parameter estimation.

---

In [ ]:
input('Press Enter key to start Extrinsic Calibration...')

In [ ]:
# Capture specifications
blob_count = 3 # Number of expected markers
delay_time = 10.0 # In seconds
capture_time = 30.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(delay_time, synchronizer):
    sys.exit() # Capture request failed!

In [3]:
verbose = False

print('[SERVER] Waiting delay...')
time.sleep(delay_time)

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Receiving messages
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

# Post-processing
for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 

        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message
        
        # The message is composed as:
        # [u, v, A] (for each blob detected) + [u_min, y_min, PTS, IID] 
        IID = message[-1] # Image Index
        PTS = message[-2] # Presentation Timestamp
        
        # Valid message must have (3 * blob_count + 4) total elements
        if message.size !=  3 * blob_count + 4: 
            if message.size == 4: 
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-4].reshape(-1, 3) # All but last 4 elements

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

[SERVER] Extrinsic Calibration info sent
[SERVER] Extrinsic Calibration confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!
[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


In [4]:
# Calibrate multiple view
wand_distances = np.array([5e-2, 10e-2, 15e-2]) # In meters

wand_blobs = [client.synchronizer.sync_blobs for client in server.clients]

if not server.multiple_view.calibrate(wand_blobs, wand_distances):
    sys.exit() # Calibration failed! 

In [5]:
# Create the Scene Viewer
scene = Viewer3D(title='Calibrated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

In [6]:
# Perform bundle adjustment
n_observations = 72
server.multiple_view.bundle_adjustment(wand_blobs, wand_distances, n_observations)

In [7]:
# Create the Scene Viewer
scene = Viewer3D(title='Adjusted Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Reference Update

Right after the Extrinsic Calibration, a new reference will be set to be the new scene's canonical frame. For that, a new reference will be requested.

---

In [ ]:
input('Press Enter key to start Reference Update...')

In [ ]:
# Capture specifications
blob_count = 3 # Number of expected markers
delay_time = 1.0 # In seconds
capture_time = 1.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(delay_time, synchronizer):
    sys.exit() # Capture request failed!

In [8]:
verbose = False

print('[SERVER] Waiting delay...')
time.sleep(delay_time)

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

# Post-processing
for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 

        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message
        
        # The message is composed as:
        # [u, v, A] (for each blob detected) + [u_min, y_min, PTS, IID] 
        IID = message[-1] # Image Index
        PTS = message[-2] # Presentation Timestamp
        
        # Valid message must have (3 * blob_count + 4) total elements
        if message.size !=  3 * blob_count + 4: 
            if message.size == 4: 
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-4].reshape(-1, 3) # All but last 4 elements

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

[SERVER] Reference Update info sent


[SERVER] Reference Update confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!
[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


In [9]:
# Measured distances between perpendicularly matched marker distances
# Distances: [D_x, D_y]
wand_distances = np.array([20e-2, 10e-2]) # In meters

# Triangulation pair
pair = (0, 2) # Diagonal pairs seems to produce more stable results

wand_blobs = [server.clients[ID].synchronizer.async_blobs for ID in pair]

# Update reference
server.multiple_view.update_reference(wand_blobs, wand_distances, pair)

In [10]:
# Create the Scene Viewer
scene = Viewer3D(title='Updated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Add new reference
scene.add_frame(np.eye(4), 'Reference', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Standard Capture

With all calibration done, a standard capture routine can be requested for the arena's usual operation.

---

In [ ]:
input('Press Enter key to start Standard Capture...')

In [17]:
# Capture specifications
blob_count = 1 # Number of expected markers
delay_time = 10.0 # In seconds
capture_time = 20.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(delay_time, synchronizer):
    sys.exit() # Capture request failed!

[SERVER] Capture info sent
[SERVER] Capture confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


In [18]:
verbose = False

print('[SERVER] Waiting delay...')
time.sleep(delay_time)

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

triangulation = 0
sync_triangulated_markers = np.empty((server.clients[0].synchronizer.sync_PTS.size, blob_count, 3))
all_triangulated_markers = []

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]}):')

    # Decode message
    try:
        message = np.frombuffer(message_bytes, dtype=np.float32)

    except:
        if verbose: print('> Couldn\'t decode message')

        continue # Jump to wait for the next message

    # Empty message
    if not message.size:
        if verbose: print('\tEmpty message')

        continue # Jump to wait for the next message

    # The message is composed as:
    # [u, v, A] (for each blob detected) + [u_min, y_min, PTS, IID] 
    IID = message[-1] # Image Index
    PTS = message[-2] # Presentation Timestamp
    
    # Valid message must have (3 * blob_count + 4) total elements
    if message.size !=  3 * blob_count + 4: 
        if message.size == 4: 
            if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

        else: 
            if verbose: 
                print(f'\tWrong blob count or corrupted message')
                print(f'Corrupted Message: {message}')

        continue # Jump to the next message

    # Extracting blob data (coordinates & area)
    blob_data = message[:-4].reshape(-1, 3) # All but last 4 elements

    # Extracting centroids
    blob_centroids = blob_data[:,:2] # Ignoring their area

    # Undistorting blobs centroids
    undistorted_blobs = client.camera.undistort_points(blob_centroids)          

    # Print blobs
    if verbose:
        print(f'\tDetected Blobs - {PTS :.3f} s')
        print('\t' + str(blob_data).replace('\n', '\n\t'))

    # Save data
    valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

    if verbose: 
        if valid_data:
            print('\tData Accepted!')
        else:
            print('\tData Refused!')

    # Check for available interpolated data
    available = []

    synchronizers = [c.synchronizer for c in server.clients]
    for ID, S in enumerate(synchronizers):
        # Is there interpolated data? Non-interpolated blobs are negative!
        if np.any(S.sync_blobs[triangulation] >= 0): 
            available.append(ID)

    # If at least two cameras have interpolated data, triangulate 
    if len(available) >= 2:
        # Get the first two available
        pair = available[:2]

        blob_pair = [synchronizers[pair[0]].sync_blobs[triangulation],
                     synchronizers[pair[1]].sync_blobs[triangulation]]
        
        triangulated_markers = server.multiple_view.triangulate_by_pair(pair, blob_pair)
        all_triangulated_markers.append(triangulated_markers)

        # Update last triangulation flag
        if triangulation < sync_triangulated_markers.shape[0] - 1:
            triangulation += 1

all_triangulated_markers = np.hstack(all_triangulated_markers)

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


# Capture Profile

---

In [ ]:
# Create the Scene Viewer
scene = Viewer3D(title='Capture Profile', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Add new reference
scene.add_frame(np.eye(4), 'Reference', axis_size=0.4)

# Add triangulated markers to the scene
scene.add_points(all_triangulated_markers, f'Markers')

# Plot scene
scene.figure.show(renderer='notebook_connected')